<a href="https://colab.research.google.com/github/saritmaitra/Natural-Gas_Paul/blob/master/Sarit_workbench_HP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Installation of ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
# do it for only one time


### Install Dash
!pip install dash==1.4.1 
!pip install dash-html-components==0.13.2
!pip install dash-core-components==0.39.0
!pip install dash-bootstrap-components
!pip install dash-table==3.1.7 
!pip install dash-daq==0.2.1
!pip install pyforest
!pip install quandl
!pip install fredapi
!pip install EIA_python
!pip install ffn

--2020-04-08 11:51:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.221.126.62, 52.4.202.19, 34.232.135.84, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.221.126.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.13M  14.4MB/s    in 0.9s    

2020-04-08 11:51:21 (14.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: Processing /root/.cache/pip/wheels/41/11/b7/643b47546d61ba55d7f29325c4952c1639edb8798887bad458/dash-1.4.1-cp36-none-any.whl
Processing /root/.cache/pip/wheels/f6/07/8b/f262dd356b8cb359a5b3f5e4ff9d4ddb85854bd51f824ffa58/dash_renderer-1.1.2-cp36-none-any.whl
Processing /root/.cache/pip/wheels/54/27/d1/b452e9aa92aa1883543248a23707d4

In [24]:
# Generate .py file to run with flask
%%writefile Sarit_app.py
from pyforest import *
import datetime, pickle, copy
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import quandl, eia, ffn

plt.style.use('ggplot')
from statistics import variance 
from random import randint
import scipy as sp
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve, auc, classification_report, accuracy_score, roc_auc_score, mean_squared_error
from xgboost import XGBClassifier, XGBRegressor, plot_importance

from fredapi import Fred

from termcolor import colored
from sklearn.model_selection import cross_val_score, KFold, cross_validate, train_test_split, TimeSeriesSplit
import xgboost as xgb

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table
import dash_bootstrap_components as dbc

app = dash.Dash(__name__)
suppress_callback_exceptions=True


# Henry Hub Spot Price, Daily (WEEKLY RELEASE )

print('\033[4mHenry Hub Natural Gas Spot Price, Daily (Dollars per Million Btu)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    hh_spot = pd.DataFrame(series_search)
    return hh_spot

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.RNGWHHD.D'
      hh_spot = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(hh_spot))
      return hh_spot;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)

hh_spot = main()
hh_spot = hh_spot.rename({'Henry Hub Natural Gas Spot Price, Daily (Dollars per Million Btu)': 'hh_spot'}, axis = 'columns')
hh_spot = hh_spot.reset_index()
hh_spot['index'] = pd.to_datetime(hh_spot['index'].str[:-3], format='%Y %m%d')
hh_spot['Date']= pd.to_datetime(hh_spot['index']) 
hh_spot.set_index('Date', inplace=True) # setting index column
hh_spot = hh_spot.loc['2000-01-01':,['hh_spot']] # setting date range
hh_spot = hh_spot.astype(float)
#print(hh_spot) 
print('\n')

# Natural Gas continuous contract#1

print('\033[4mNatural Gas continuous contract 1\033[0m')
NG = quandl.get("CHRIS/CME_NG1", authtoken="LSQpgUzwJRoF667ZpzyL") # natural gas continuous contract 1
NG = NG.loc['2000-01-01':,]


# SPOT/OPN/CLOSE/ADJ CLOSE/HIGH/LOW/VOLUME

print('\033[4mCurrent spot price\033[0m')
print(round(hh_spot,2).tail(1))
print('\n')
print('\033[4mOpen price\033[0m')
print(round(NG['Open'],2).tail(1))
print('\n')
print('\033[4mClose price\033[0m')
print(round(NG['Last'],2).tail(1))
print('\n')
print('\033[4mAdj Close price\033[0m')
print(round(NG['Settle'],2).tail(1))
print('\n')
print('\033[4mHigh price\033[0m')
print(round(NG['High'],2).tail(1))
print('\n')
print('\033[4mLow price\033[0m')
print(round(NG['Low'],2).tail(1))
print('\n')
print('\033[4mVolume\033[0m')
print(round(NG['Volume'],2).tail(1))
print('\n')

import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x=NG.index,
                open=NG['Open'],
                high=NG['High'],
                low=NG['Low'],
                close=NG['Last'])])
fig.show()
print('\n')

# 4 WEEKS's OPEN/CLOSE/HIGH/LOW PRICE
ng = round(NG[['Open', 'Last', 'High', 'Low']],2).copy()
ng.tail(20)


# STATISTICS 1

perf = NG['Last'].calc_stats()
print('\n')
perf.display()
print('\n')

# STATISTICS 2

# Calculate the daily percentage change which is daily return
#print('\033[1m' + 'daily percentage change' + '\033[1m')
daily_ret = NG['Last'].pct_change().fillna(0)
mean_return = daily_ret.mean()
return_stdev = daily_ret.std()

print('Average daily return %:', round(mean_return,2)*100)
print('Average Volatility %:', round(return_stdev,2)*100)

print ('Annualized average return %:', round(mean_return * 252,2)*100)
print('Annualized volatility %:', round(return_stdev * np.sqrt(252), 2)*100)
print('\n')


print('\033[4mMaximum Drawdown\033[0m')

# Using a trailing 186 trading day window
window = 186

# Calculate the max drawdown in the past window days for each day in the series.
# using min_periods=1, we want to let the first 186 days data have an expanding window
roll_max = NG['Last'].rolling(min_periods=1, window=window).max()

# Calculate daily draw-down from rolling max
daily_drawdown = NG['Last']/roll_max - 1.0
#print('\033[4mMaximum Daily Drawdown\033[0m')
#print(daily_drawdown)
#print('\n')

# Next we calculate the minimum (negative) daily drawdown in that window.
# using min_periods=1 to allow the expanding window
#print('\033[4mMin Daily Drawdown\033[0m')
min_daily_drawdown = daily_drawdown.rolling(min_periods=1, window=window).min()
#print(min_daily_drawdown)
print('\n')

# Plot the results
daily_drawdown.plot(figsize=(20,8))
min_daily_drawdown.plot()
#plt.title('Orange -> Daily running 186-day drawdown; Blue -> Max. experienced 186-day drawdown in the past year')
plt.title('Daily draw-down')
plt.grid()
plt.show()
print('\n')


print('\033[4mAverage Daily Sharpe %\033[0m')
sharpe_ratio = daily_ret.mean() / daily_ret.std()
print('Sharpe Ratio:', round(sharpe_ratio,2)*100)
print('\n')

print('\033[4mAverage Annual Sharpe %\033[0m')
an_sharpe_ratio = (252**0.5) * sharpe_ratio # annualised sharpe ratio
an_sharpe_ratio = round(an_sharpe_ratio,2)
print('Annualized Sharpe %:', an_sharpe_ratio*100)


# STATISTICS 3

print('\033[4mProbability of +/-(1%); +/-(3%); +/-%(5) change in price (Data -> 2000- till date)\033[0m')

print ("The probability of price changes between 1%% and -1%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.01) & (daily_ret < 0.01)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes between 3%% and -3%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.03) & (daily_ret < 0.03)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes between 5%% and -5%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.05) & (daily_ret < 0.05)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes more than 5%% is %1.2f%%" % 
       (100*daily_ret[daily_ret > 0.05].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes less than -5%% is %1.2f%%" % 
       (100*daily_ret[daily_ret < -0.05].shape[0] / daily_ret.shape[0]))
print('\n')

print('\033[4mMinimum price [2000- till date]\033[0m')
print(round(NG['Last'].min(),2), NG['Last'].idxmin());
print('\n')
print('\033[4mMaximum price [2000- till date]\033[0m')
print(round(NG['Last'].max(),2), NG['Last'].idxmax());
print('\n')

print('\033[4mMinimum daily % return [2000- till date]\033[0m')
print(round(daily_ret.min(),2)*100, daily_ret.idxmin()); 
print('\n')
print('\033[4mMaximum daily % return [2000- till date]\033[0m')
print(round(daily_ret.max(),2)*100, daily_ret.idxmax());
print('\n')


# DATA PIPELINE

# Cushing, OK WTI Spot Price FOB, Daily
print('\033[4mWTI Spot Price FOB, Daily (Dollars per Barrel)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    wti = pd.DataFrame(series_search)
    return wti

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='PET.RWTC.D'
      wti = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(wti))
      return wti;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)

wti = main()
wti = wti.rename({'Cushing, OK WTI Spot Price FOB, Daily (Dollars per Barrel)': 'wti'}, axis = 'columns')
wti = wti.reset_index()
wti['index'] = pd.to_datetime(wti['index'].str[:-3], format='%Y %m%d')
wti['Date']= pd.to_datetime(wti['index']) 
wti.set_index('Date', inplace=True) # setting index column
wti = wti.loc['2000-01-01':,['wti']] # setting date range
wti = wti.astype(float)
#print(wti) 
#print('\n')

## GDP data
print('\033[4mUS GDP Data\033[0m')
fred = Fred(api_key='59c798fcda5850ef874412d269fd2378')
gdp = fred.get_series_as_of_date('GDP', '2020-01-31')
gdp = gdp.rename({'date': 'Date', 'value': 'gdp'}, axis = 'columns')
gdp.set_index('Date', inplace=True) # setting index column
gdp = gdp.loc['2000-01-01':'2020-02-02',['gdp']] # setting date range
gdp.replace({pd.NaT: "0"}, inplace=True) # replacing NaT with 0
gdp = gdp.ffill().bfill()
gdp = gdp.reset_index().drop_duplicates('Date').set_index('Date') # dropping duplicate concurrence
gdp = pd.DataFrame(gdp)
gdp = gdp.astype(float)
gdp = gdp.resample('B').ffill()
gdp = gdp/4
#print(gdp)
#print('\n')

# U.S. Natural Gas Marketed Production, Monthly
print('\033[4mNatural Gas Marketed Production, Monthly (Million Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    prod = pd.DataFrame(series_search)
    return prod

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9050US2.M'
      prod = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(prod))
      return prod;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)

prod = main()
prod = prod.rename({'U.S. Natural Gas Marketed Production, Monthly (Million Cubic Feet)': 'prod'}, axis = 'columns')
prod = prod.reset_index()
prod['Date']= pd.to_datetime(prod['index']) 
prod.set_index('Date', inplace=True) # setting index column
prod = prod.loc['2000-01-01':,['prod']] # setting date range
prod = prod.astype(float)
prod = prod.resample('B').ffill()
prod = prod/21
#print(prod)
#print('\n')


# U.S. Natural Gas Gross Withdrawals, Monthly
print('\033[4mNatural Gas Gross Withdrawals, Monthly (Million Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    withdrawals = pd.DataFrame(series_search)
    return withdrawals

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9010US2.M' 
      withdrawals = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(withdrawals))
      return withdrawals;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)

withdrawals = main()
withdrawals = withdrawals.rename({'U.S. Natural Gas Gross Withdrawals, Monthly (Million Cubic Feet)': 'withdrawals'}, axis = 'columns')
withdrawals = withdrawals.reset_index()
withdrawals['Date']= pd.to_datetime(withdrawals['index']) 
withdrawals.set_index('Date', inplace=True) # setting index column
withdrawals = withdrawals.loc['2000-01-01':,['withdrawals']] # setting date range
withdrawals = withdrawals.astype(float)
withdrawals = withdrawals.resample('B').bfill().ffill()
withdrawals = withdrawals/21
#print(withdrawals)
#print('\n')

# U.S. Natural Gas Underground Storage Volume, Monthly
print('\033[4mNatural Gas Underground Storage Volume, Monthly\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    storage = pd.DataFrame(series_search)
    return storage

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N5030US2.M '
      storage = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(storage))
      return storage;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)

storage = main()
storage = storage.rename({'U.S. Natural Gas Underground Storage Volume, Monthly (Million Cubic Feet)': 'storage'}, axis = 'columns')
storage = storage.reset_index()
storage['Date']= pd.to_datetime(storage['index']) 
storage.set_index('Date', inplace=True) # setting index column
storage = storage.loc['2000-01-01':,['storage']] # setting date range
stoarge = storage.astype(float)
storage = storage.resample('B').bfill().ffill()
storage = storage/21
#print(storage)
#print('\n')

# U.S. Natural Gas Exports, Monthly
print('\033[4mNatural Gas Exports, Monthly (Million Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    export = pd.DataFrame(series_search)
    return export

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9133US2.M'
      export = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(export))
      return export;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)

export = main()
export = export.rename({'Liquefied U.S. Natural Gas Exports, Monthly (Million Cubic Feet)': 'export'}, axis = 'columns')
export = export.reset_index()
export['Date']= pd.to_datetime(export['index']) 
export.set_index('Date', inplace=True) # setting index column
export = export.loc['2000-01-01':,['export']] # setting date range
export = export.astype(float)
export = export.resample('B').bfill().ffill()
export = export/21
#print(export)
#print('\n')

# natural gas total consumption (monthly))
print('\033[4mNatural Gas Total Consumption, Monthly (Million Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    consumption = pd.DataFrame(series_search)
    return consumption

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9140US2.M'
      consumption = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(consumption))
      return consumption;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)

consumption = main()
consumption = consumption.rename({'U.S. Natural Gas Total Consumption, Monthly (Million Cubic Feet)': 'consumption'}, axis = 'columns')
consumption = consumption.reset_index()
consumption['Date']= pd.to_datetime(consumption['index']) 
consumption.set_index('Date', inplace=True) # setting index column
consumption = consumption.loc['2000-01-01':,['consumption']] # setting date range
consumption = consumption.astype(float)
consumption = consumption.resample('B').ffill().bfill()
consumption = consumption/21
#print(consumption)
#print('\n')

# New York Harbor No. 2 Heating Oil Spot Price FOB, Monthly
print('\033[4mHeating Oil Spot Price FOB, Daily (Dollars per Gallon)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    heatingoil = pd.DataFrame(series_search)
    return heatingoil

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='PET.EER_EPD2F_PF4_Y35NY_DPG.D'
      heatingoil = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(heatingoil))
      return heatingoil;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)

heatingoil = main()
heatingoil = heatingoil.rename({'New York Harbor No. 2 Heating Oil Spot Price FOB, Daily (Dollars per Gallon)': 'heatingoil'}, axis = 'columns')
heatingoil = heatingoil.reset_index()
heatingoil['index'] = pd.to_datetime(heatingoil['index'].str[:-3], format='%Y %m%d')
heatingoil['Date']= pd.to_datetime(heatingoil['index']) 
heatingoil.set_index('Date', inplace=True) # setting index column
heatingoil = heatingoil.loc['2000-01-01':,['heatingoil']] # setting date range
heatingoil = heatingoil.astype(float)
#print(heatingoil)
#print('\n')

# natural gas import dat
#print('\033[4mNatural Gas Imports, Monthly (Million Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    g_import = pd.DataFrame(series_search)
    return g_import

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9100US2.M'
      g_import = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(g_import))
      return g_import;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
  
g_import = main()
g_import = g_import.rename({'U.S. Natural Gas Imports, Monthly (Million Cubic Feet)': 'g_import'}, axis = 'columns')
g_import = g_import.reset_index()
g_import['Date']= pd.to_datetime(g_import['index']) 
g_import.set_index('Date', inplace=True) # setting index column
g_import = g_import.loc['2000-01-01':,['g_import']] # setting date range
g_import = g_import.astype(float)
g_import = g_import.resample('B').ffill().bfill()
g_import = g_import/21
#print(g_import)
#print('\n')

# U.S. Natural Gas Rotary Rigs in Operation, Monthly
print('\033[4mNatural Gas Rotary Rigs in Operation, Monthly (Number of Elements)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    drilling = pd.DataFrame(series_search)
    return drilling

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='PET.E_ERTRRG_XR0_NUS_C.M '
      drilling = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(drilling))
      return drilling;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
  
drilling = main()
drilling = drilling.rename({'U.S. Natural Gas Rotary Rigs in Operation, Monthly (Number of Elements)': 'drilling'}, axis = 'columns')
drilling = drilling.reset_index()
drilling['Date']= pd.to_datetime(drilling['index']) 
drilling.set_index('Date', inplace=True) # setting index column
drilling = drilling.loc['2000-01-01':,['drilling']] # setting date range
drilling = drilling.astype(float)
drilling = drilling.resample('B').ffill().bfill()
drilling = drilling/21
#print(drilling)
#print('\n')

# Heating Degree Days U.S. Average , Monthly
print('\033[4mHeating Degree Days U.S. Average , Monthly (total degree days)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    HDD_US = pd.DataFrame(series_search)
    return HDD_US

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='STEO.ZWHDPUS.M'
      HDD_US = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(HDD_US))
      return HDD_US;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
  
HDD_US = main()
HDD_US = HDD_US.rename({'Heating Degree Days U.S. Average , Monthly (total degree days)': 'HDD_US'}, axis = 'columns')
HDD_US = HDD_US.reset_index()
HDD_US['Date']= pd.to_datetime(HDD_US['index']) 
HDD_US.set_index('Date', inplace=True) # setting index column
HDD_US = HDD_US.loc['2000-01-01':,['HDD_US']] # setting date range
HDD_US = HDD_US.astype(float)
HDD_US = HDD_US.resample('B').ffill().bfill()
HDD_US = HDD_US/21
#print(HDD_US)
#print('\n')

# Cooling Degree-Days, United States, Monthly
print('\033[4mCooling Degree-Days, United States, Monthly (Number)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    CDD_US = pd.DataFrame(series_search)
    return CDD_US

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='TOTAL.ZWCDPUS.M'
      CDD_US = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(CDD_US))
      return CDD_US;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
  
CDD_US = main()
CDD_US = CDD_US.rename({'Cooling Degree-Days, United States, Monthly (Number)': 'CDD_US'}, axis = 'columns')
CDD_US = CDD_US.reset_index()
CDD_US['Date']= pd.to_datetime(CDD_US['index']) 
CDD_US.set_index('Date', inplace=True) # setting index column
CDD_US = CDD_US.loc['2000-01-01':,['CDD_US']] # setting date range
CDD_US = CDD_US.astype(float)
CDD_US = CDD_US.resample('B').ffill().bfill()
CDD_US = CDD_US/21
#print(CDD_US)
#print('\n')

# SUPERVISED ML

# merging data frames
merge1 = NG.join(gdp, how='left').ffill().bfill()
merge2 = merge1.join(wti, how = 'left')
merge3 = merge2.join(withdrawals, how = 'left').ffill().bfill()
merge4 = merge3.join(storage, how = 'left').ffill().bfill()
merge5 = merge4.join(prod, how = 'left').ffill().bfill()
merge6 = merge5.join(consumption, how = 'left').ffill().bfill()
merge7 = merge6.join(heatingoil, how ='left').ffill().bfill()
merge8 = merge7.join(drilling, how = 'left').ffill().bfill()
merge9 = merge8.join(export, how = 'left').ffill().bfill()
merge10 = merge9.join(g_import, how = 'left').ffill().bfill()
merge11 = merge10.join(HDD_US, how = 'left').ffill().bfill()
merge12 = merge11.join(CDD_US, how = 'left').ffill().bfill()

# feature engineering
merge12['day_of_week'] = merge12.index.dayofweek
merge12['day_of_month'] = merge12.index.day
merge12['quarter'] = merge12.index.quarter
merge12['month'] = merge12.index.month
merge12['year'] = merge12.index.year

merge13 = merge12.apply(copy.deepcopy)
#print(merge13)

# Technical indicators
merge13['daily_ret'] = merge13['Open'].pct_change()
merge13['volatility'] = merge13['daily_ret'].rolling(252).std()*(252**0.5)
merge13['price_diff'] = merge13['Last'] - merge13['Open']
merge13["vol_increment"] = merge13['Volume'].diff()
merge13["vol_rel_increment"] = merge13['Volume'].diff() / merge13['Volume']

sma1 = 20
sma2 = 100
merge13["sma1"] = merge13['Open'].rolling(sma1).mean()
merge13["sma2"] = merge13['Open'].rolling(sma2).mean()

merge13['ema_12'] = merge13['Open'].ewm(span=10).mean()
merge13['ema_26'] = merge13['Open'].ewm(span=26).mean()
merge13['ROC'] = ((merge13['Open'] - merge13['Open'].shift(5)) / (merge13['Open'].shift(5)))*100

delta = merge13['Open'].diff()
window = 14
up_days = delta.copy()
up_days[delta<=0]=0.0
down_days = abs(delta.copy())
down_days[delta>0]=0.0
RS_up = up_days.rolling(window).mean()
RS_down = down_days.rolling(window).mean()
merge13['rsi'] = 100-100/(1+RS_up/RS_down)
merge13['macd'] = merge13['ema_12'] - merge13['ema_26']

#print('\033[4mMerged dataframe\033[0m')
#print(df.info())
#print('\n')


#print('\033[4mProcessed dataframe\033[0m')
df = merge13.apply(copy.deepcopy)
#print(df)
#print(df.info())
#print('\n')


#fig = plt.figure(figsize=(20,6))
#sns.lineplot(x='day_of_week', y= 'Open', data=df)
#plt.title('Natural gas continuous contract 1- day of the week opening price from 2000 to till date')
#plt.show()
#print('\n')

#fig = plt.figure(figsize=(20,6))
#sns.lineplot(x='month', y= 'Open', data=df);
#plt.title('Natural gas continuous contract 1- Monthly opening price from 2000 to till date', fontsize = 12)
#plt.show()
#print('\n')


## WINDOW FEATUIRE

df['t-2'] = df['Open'].shift(3)
df['t-1'] = df['Open'].shift(2)
df['t+1'] = df['Open'].shift(1)
#df.head()

# ROLLING WINDOW STATISTICS

width = 3
shifted = df['Open'].shift(width - 1)
roll_window = shifted.rolling(window=width)
df['min'] = roll_window.min()
df['mean'] = roll_window.mean()
df['max'] = roll_window.max()
#df.head()

# EXPANDING WINDOW STATISTICS

exp_window = df['Open'].expanding()
df['min_exp'] = exp_window.min()
df['mean_exp'] = exp_window.mean()
df['max_exp'] = exp_window.max()
#df.head()

# BUY / SELL SIGNAL 

#%matplotlib inline 

""" run this section just once """

# Target Variable
print('\033[4mThis is a classification variable, average price will go either up or down the next day.\033[0m')
#This will be a classification variable, if the average price will go either up or down the next day.  
#The target will be forecasting the difference between today’s price and tomorrow’s price (which is unkonwn).

df['target'] = (df['sma1'].shift(-1) - df['sma1'])
#df.fillna(0,inplace=True)
df['target'].tail()
#print('\n')

#The target variables will be transformed for binary classification.
def getBinary(val):
    if val > 0:
      return 1
    else:
      return 0
df['next_day_direction'] = df["target"].apply(getBinary)
print('\033[4mThe target variables transformed for binary classification. A positive change in the value of prices classified as 1 and a non-positive change as 0.\033[0m')

#print(df.tail())
#print('\n')

y = df['next_day_direction'].copy()
x = df.copy().drop(['next_day_direction', 
                    'target', 
                    'Open', 
                    'High', 
                    'Low', 
                    'Volume', 
                    'Settle', 
                    'Change', 
                    'Previous Day Open Interest',
                    'day_of_week', 
                    'day_of_month', 
                    'HDD_US', 
                    'quarter', 
                    'year'], axis=1)

X = np.array(x)
y = np.array(y)
tscv = TimeSeriesSplit()
#print(tscv)
TimeSeriesSplit(max_train_size=0.67, n_splits=5)
for train_index, test_index in tscv.split(X):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  # convert the numpy arrays into Pandas DataFrames so they are easier to explore and work with
  X_train = pd.DataFrame(data=X_train, index=train_index)
  y_train = pd.Series(data=y_train, index=train_index)
  X_test = pd.DataFrame(data=X_test, index=test_index)
  y_test = pd.Series(data=y_test, index=test_index)

eval_set = [(X_train, np.ravel(y_train)), (X_test, np.ravel(y_test))]


#xgb_clf1 = XGBClassifier()

# Hyperparameter optimization
#xgb = xgb_clf1.fit(X_train, y_train, 
#                   early_stopping_rounds=10,  
#                   eval_metric="logloss", 
#                   eval_set=[(X_test, y_test)])

# scores
#from  sklearn.metrics import log_loss
#log_train = log_loss(y_train, xgb.predict_proba(X_train)[:,1])
#log_valid = log_loss(y_test, xgb.predict_proba(X_test)[:,1])


#print('\n-----------------------')
#print('  logloss train: %.5f'%log_train)
#print('  logloss valid: %.5f'%log_valid)
#print('-----------------------')

#print('\nModel parameters...')
#print(xgb.get_params())



# Set the hyperparameters

base_score = 0.5
colsample_bylevel = 1
colsample_bytree = 1
gamma = 0
learning_rate = 0.1
max_depth = 3
min_child_weight = 1
n_estimators = 100
subsample =1
colsample_bytree = 0.5
gamma = 1
random_state = 42


xgb_clf1 = XGBClassifier(learning_rate= learning_rate,
                         base_score = base_score,
                         max_depth= max_depth,
                         min_child_weight= min_child_weight,
                         n_estimators= n_estimators,
                         subsample =subsample,
                         colsample_bytree = colsample_bytree,
                         gamma = gamma,
                         random_state = random_state)


xgb_clf1.fit(X_train, y_train, 
             eval_metric = 'auc', 
             eval_set = eval_set,
             early_stopping_rounds = 10, 
             verbose = 10)


# Clasifier performance
#%matplotlib inline 
print('\033[4mModel performance :: Quality of Classifier\033[0m')
plt.rcParams['figure.figsize'] = 15, 5 
plt.rcParams['font.size'] = 12

history = xgb_clf1.evals_result_
x_axis = range(len(history['validation_0']['auc']))
plt.plot(x_axis, history['validation_0']['auc'], label = 'Train')
plt.plot(x_axis, history['validation_1']['auc'], label = 'Test')
plt.legend(loc = 'best')
plt.ylabel('AUC')
plt.title('Xgboost AUC')
plt.show()

# we can then access the best number of tree and use it later for prediction
#print('best iteration', xgb_clf1.best_ntree_limit)
#print('\n')


# Feature importance
#plt.rcParams['figure.figsize'] = [15, 8]
#plot_importance(xgb_clf1)
#print('\n')

# print the model's performance
ntree_limit = xgb_clf1.best_ntree_limit
xgb_clf1.predict_proba(X_test[:5], ntree_limit = ntree_limit)

# Area under Curve

pred_prob = xgb_clf1.predict_proba(X_test, ntree_limit = ntree_limit)[:, 1]
print('Area under ROC curve:', round(roc_auc_score(y_test, pred_prob)*100, 2))
print('\n')


# print the model's performance
ntree_limit = xgb_clf1.best_ntree_limit
#print('\033[4mNext day Open price direction\033[0m')
pred = xgb_clf1.predict(X_test, ntree_limit = ntree_limit)
pred = pd.DataFrame(pred)
print(pred.tail())
print('\n')

#print('\033[4mProbability of prediction results\033[0m')
#pred_prob = pd.DataFrame(pred_prob)
#print(round(pred_prob.tail(),2))
#print('\n')


# For each day where open price > 0, we set the signal 1 means sell  and < 0, we set signal 0, to buy 

print('\033[4mFor each day where open price = 0 = red arrow head  and 1 = green arrow head \033[0m')
buys = df.loc[df['next_day_direction'] == 0]
sells = df.loc[df['next_day_direction'] == 1]

# Plot 
fig = plt.figure(figsize=(20,10))
plt.plot(df.index, df['Open'], color = 'b', lw=2., label='Last')

# Plot the buy and sell signals on the same plot
plt.plot(buys.index, df.loc[buys.index]['Open'], 'v', markersize=10, color='r')
plt.plot(sells.index, df.loc[sells.index]['Open'], '^', markersize=10, color='g')
plt.ylabel('Price')
plt.xlabel('Date')
plt.title('Buy (Red) and sell (Green) signals')
plt.legend(loc=0)

# Display everything
plt.show()
print('\n')


# Last 5 Buy & Sell signals
print('\033[4mLast 5 Buy signal \033[0m')
print(round(df.loc[buys.index], 2)['Open'].tail())
print('\n')
print('\033[4mLast 5 Sell signal \033[0m')
print(round(df.loc[sells.index], 2)['Open'].tail())
print('\n')


# assigning date to the predicted values
df['Forecast'] = np.nan
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day 

for i in pred:
  next_date = datetime.datetime.fromtimestamp(next_unix)
  next_unix += one_day
  df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]

#create an index of just the date portion of our index (this is the slow step)
ts_days = pd.to_datetime(df['Forecast'].index.date)

#create a range of business days over that period
bdays = pd.bdate_range(start=df['Forecast'].index[0].date(), end=df['Forecast'].index[-1].date())

#Filter the series to just those days contained in the business day range.
df['Forecast'] = df.Forecast[ts_days.isin(bdays)]
print('\033[4mNext day Open price direction\033[0m')
df['Forecast'].tail()
#print(df)



## Moving averages crossover [2 months and 1 year look back]
merge14 = merge12.apply(copy.deepcopy)

# feature engineering
merge14['day_of_week'] = merge14.index.dayofweek
merge14['day_of_month'] = merge14.index.day
merge14['quarter'] = merge14.index.quarter
merge14['month'] = merge14.index.month
merge14['year'] = merge14.index.year

# Technical indicators
merge14['daily_ret'] = merge14['Last'].pct_change()
merge14["Last_1"] = merge14['Last'].shift(1)
merge14["Last_incr"] = merge14['Last'] - merge14['Last'].shift(1)
merge14['price_diff'] = merge14['Last'] - merge14['Open']
merge14["vol_increment"] = merge14['Volume'].diff()
merge14["vol_rel_increment"] = merge14['Volume'].diff() / merge14['Volume']

sma1 = 42
sma2 = 252
merge14["sma1"] = merge14['Last'].rolling(sma1).mean().fillna(0)
merge14["sma2"] = merge14['Last'].rolling(sma2).mean().fillna(0)
merge14['ema42'] = merge14['Last'].ewm(span=42).mean().fillna(0)
merge14['ema252'] = merge14['Last'].ewm(span=252).mean().fillna(0)

merge14['ema_12'] = merge14['Last'].ewm(span=10).mean().fillna(0)
merge14['ema_26'] = merge14['Last'].ewm(span=26).mean().fillna(0)
merge14['ROC'] = ((merge14['Last'] - merge14['Last'].shift(5)) / (merge14['Last'].shift(5)))*100

delta = merge14['Last'].diff()
window = 14
up_days = delta.copy()
up_days[delta<=0]=0.0
down_days = abs(delta.copy())
down_days[delta>0]=0.0
RS_up = up_days.rolling(window).mean()
RS_down = down_days.rolling(window).mean()
merge14['rsi'] = 100-100/(1+RS_up/RS_down)
merge14['macd'] = merge14['ema_12'] - merge14['ema_26']

df1 = merge14.apply(copy.deepcopy)

# moving average crossover :: sma 252 is > sma 42
df1['target'] = np.where(df1['sma1'] > df1['sma2'], 1,0)
df1['target'].tail()
#print('\n')

def getBinary(val):
    if val>0:
        return 1
    else:
        return 0
df1['next_day_direction'] = df1["target"].apply(getBinary)


print('\033[4mSell-> Shorter MA crosses above the longer-term MA (Golden cross)\033[0m')
print('\033[4mBuy-> Shorter MA crosses below the longer-term MA(Dead cross)\033[0m')
buys = df1.loc[df1['next_day_direction'] == 0]
sells = df1.loc[df1['next_day_direction'] == 1]
print('\n')

print("\033[4mBuys data to checking\033[0m")
print(round(df1.loc[buys.index]['Last'],2).tail())
print('\n')
print("\033[4mSells data to checking\033[0m")
print(round(df1.loc[sells.index]['Last'],2).tail())


# SMA crossover Plot 

fig = plt.figure(figsize=(20,6))
plt.plot(merge14['Last'], color='gray', label='Last')
# Plot the buy and sell signals on the same plot
#  When the shorter-term MA crosses above the longer-term MA (Golden cross), it's a buy signal, it indicates that the trend is shifting up. 
#  When the shorter-term MA crosses below the longer-term MA, it's a sell signal (dead/death cross), it indicates that the trend is shifting down. 
plt.plot(merge14['sma1'].dropna(), color='r', label = 'sma42')
plt.plot(merge14['sma2'].dropna(), color='g', label = 'sma252')
plt.ylabel('Price')
plt.xlabel('Date')
plt.suptitle('SMA crossover')
plt.title('Buy -> Shorter MA crosses below longer MA (Deadcross); Sell -> Shorter MA crosses above longer MA (Golden cross)')
plt.legend(loc=0)

# Display everything
plt.show()
print('\n')


# 5 days look-ahead Open price

## Technical indicators

merge16 = merge12.apply(copy.deepcopy)

merge16['pct_change'] = merge16['Open'].pct_change()
merge16["vol_rel_increment"] = merge16['Volume'].diff() / merge16['Volume']
merge16['std_5'] = merge16['pct_change'].rolling(5).std()
merge16['ret_5'] = merge16['pct_change'].rolling(5).mean()
merge16['sma42'] = merge16['Open'].rolling(42).mean()
merge16['sma252'] = merge16['Open'].rolling(252).mean()
merge16['ema_12'] = merge16['Open'].ewm(span=10).mean()
merge16['ema_26'] = merge16['Open'].ewm(span=26).mean()
merge16['ROC'] = ((merge16['Open'] - merge16['Open'].shift(5)) / (merge16['Open'].shift(5)))*100

delta = merge16['Open'].diff()
window = 14
up_days = delta.copy()
up_days[delta<=0]=0.0
down_days = abs(delta.copy())
down_days[delta>0]=0.0
RS_up = up_days.rolling(window).mean()
RS_down = down_days.rolling(window).mean()
merge16['rsi'] = 100-100/(1+RS_up/RS_down)
merge16['macd'] = merge16['ema_12'] - merge16['ema_26']

df4 = merge16.apply(copy.deepcopy)

# WINDOW FEATURE

df4['t-2'] = df4['Open'].shift(3)
df4['t-1'] = df4['Open'].shift(2)
df4['t+1'] = df4['Open'].shift(1)

# ROLLING WINDOW FEATURE

width = 3
shifted = df4['Open'].shift(width - 1)
roll_window = shifted.rolling(window=width)
df4['min'] = roll_window.min()
df4['mean'] = roll_window.mean()
df4['max'] = roll_window.max()

# EXPANDING WINDOW STATISTICS

exp_window = df4['Open'].expanding()
df4['min_exp'] = exp_window.min()
df4['mean_exp'] = exp_window.mean()
df4['max_exp'] = exp_window.max()

# A variable for predicting 'n' days out into the future
n = int(5) #'n=5' days
future_pred = n 

#Create another column (the target or dependent variable) shifted 'n' units up
df4['Prediction'] = df4['Open'].shift(-future_pred)

#print the new data set
#print(df4)

# settimng up unseen data
X3 = np.array(df4.drop(columns = ['Prediction',
                                  'day_of_week',
                                  'Change',
                                  'min_exp'], axis=1))
                         
#X3 = np.array (df4.drop(['Prediction'],1))
X3_forecast = X3[-future_pred:] # set X3_forecast equal to last 5, we do not have y3 values for X3_forecast
X3 = X3[:-future_pred] # remove last 5 from X3
y3 = np.array(df4['Prediction'])
y3 = y3[:-future_pred]
#print(len(X3), len(y3))

# spliting train/test data
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size = 0.33, random_state=42)

# time series cross validation
tscv = TimeSeriesSplit()
#print(tscv)
TimeSeriesSplit(max_train_size=0.67, n_splits=5)
for train_index, test_index in tscv.split(X3):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X3_train, X3_test = X3[train_index], X3[test_index]
  y3_train, y3_test = y3[train_index], y3[test_index]


eval_set = [(X3_train, np.ravel(y3_train)), (X3_test, np.ravel(y3_test))]

# importing necessary package
from xgboost import XGBRegressor
import xgboost as xgb

reg2 = xgb.XGBRegressor(objective ='reg:squarederror',
                        colsample_bytree = 0.3, 
                        learning_rate = 0.1, 
                        max_depth = 5, 
                        alpha = 10, 
                        n_estimators = 100,
                        random_state = 42)

# Create the model on train dataset
reg2.fit(X3_train, y3_train, 
         eval_metric = 'rmse', 
         eval_set = eval_set,
         early_stopping_rounds = 10, verbose = 10)


# Regression performance check
print('\033[4mModel performance :: Regression Quality\033[0m')
plt.rcParams['figure.figsize'] = 15, 5 
plt.rcParams['font.size'] = 12

history = reg2.evals_result_
x_axis = range(len(history['validation_0']['rmse']))
plt.plot(x_axis, history['validation_0']['rmse'], label = 'Train')
plt.plot(x_axis, history['validation_1']['rmse'], label = 'Test')
plt.legend(loc = 'best')
plt.ylabel('RMSE')
plt.title('Xgboost model performance')
plt.show()

# Important features
plt.rcParams['figure.figsize'] = [15, 10]
plot_importance(reg2)
print('\n')

open_confidence = reg2.score(X3_test, y3_test)
print("open_confidence %: ", round(open_confidence*100,2))

# 5 days Open prediction
forecast_pred_2 = reg2.predict(X3_forecast)
#print('\033[4mExpected Open price for next 5 days\033[0m')
#print(forecast_pred_2)
#print('\n')

rmse1 = np.sqrt(mean_squared_error(y3_test[:future_pred], forecast_pred_2[:future_pred]))
print('Test RMSE: %.2f' % rmse1)


# assigning date to the predicted values
merge16['open_pred'] = np.nan
last_date = merge16.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day 

for i in forecast_pred_2:
  next_date = datetime.datetime.fromtimestamp(next_unix)
  next_unix += one_day
  merge16.loc[next_date] = [np.nan for _ in range(len(merge16.columns)-1)] + [i]

#create an index of just the date portion of our index (this is the slow step)
ts_days = pd.to_datetime(merge16['open_pred'].index.date)

#create a range of business days over that period
bdays = pd.bdate_range(start=merge16['open_pred'].index[0].date(), end=merge16['open_pred'].index[-1].date())

#Filter the series to just those days contained in the business day range.
merge16['open_pred'] = round(merge16.open_pred[ts_days.isin(bdays)],2)

print('\033[4mExpected Open price for next 5 days\033[0m')
merge16['open_pred'].tail()


# 5 Days look ahead Close price

# Technical indicators
## 5 days look-ahead Open price

# Technical indicators
merge15 = merge12.apply(copy.deepcopy)

merge15['pct_change'] = merge15['Last'].pct_change()
merge15["vol_rel_increment"] = merge15['Volume'].diff() / merge15['Volume']
merge15['std_5'] = merge15['pct_change'].rolling(5).std()
merge15['ret_5'] = merge15['pct_change'].rolling(5).mean()
merge15['sma42'] = merge15['Last'].rolling(42).mean()
merge15['sma252'] = merge15['Last'].rolling(252).mean()
merge15['ema_12'] = merge15['Last'].ewm(span=10).mean()
merge15['ema_26'] = merge15['Last'].ewm(span=26).mean()
merge15['ROC'] = ((merge15['Last'] - merge15['Last'].shift(5)) / (merge15['Last'].shift(5)))*100

delta = merge15['Last'].diff()
window = 14
up_days = delta.copy()
up_days[delta<=0]=0.0
down_days = abs(delta.copy())
down_days[delta>0]=0.0
RS_up = up_days.rolling(window).mean()
RS_down = down_days.rolling(window).mean()
merge15['rsi'] = 100-100/(1+RS_up/RS_down)
merge15['macd'] = merge15['ema_12'] - merge15['ema_26']

df3 = merge15.apply(copy.deepcopy)

# WINDOW FEATURE

df3['t-2'] = df3['Last'].shift(3)
df3['t-1'] = df3['Last'].shift(2)
df3['t+1'] = df3['Last'].shift(1)

# ROLLING WINDOW FEATURE

width = 3
shifted = df3['Last'].shift(width - 1)
roll_window = shifted.rolling(window=width)
df3['min'] = roll_window.min()
df3['mean'] = roll_window.mean()
df3['max'] = roll_window.max()

# EXPANDING WINDOW STATISTICS

exp_window = df3['Last'].expanding()
df3['min_exp'] = exp_window.min()
df3['mean_exp'] = exp_window.mean()
df3['max_exp'] = exp_window.max()


# A variable for predicting 'n' days out into the future
n = int(5) #'n=5' days
pred_future = n 

#Create another column (the target or dependent variable) shifted 'n' units up
df3['prediction'] = df3['Last'].shift(-pred_future)

#print the new data set
#print(df4)

X2 = np.array(df3.drop(columns = ['prediction',
                                  'day_of_week',
                                  'Change',
                                  'min_exp'], axis=1))
                         
#X2 = np.array (df3.drop(['Prediction'],1))
X2_forecast = X2[-pred_future:] # set X2_forecast equal to last 5, we do not have y2 values for X2_forecast
X2 = X2[:-pred_future] # remove last 5 from X2
y2 = np.array(df3['prediction'])
y2 = y2[:-pred_future]
#print(len(X2), len(y2))

# spliting train and test set
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.33, random_state=42)

# time series cross validation 
tscv = TimeSeriesSplit()
#print(tscv)
TimeSeriesSplit(max_train_size=0.67, n_splits=5)
for train_index, test_index in tscv.split(X2):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X2_train, X2_test = X2[train_index], X2[test_index]
  y2_train, y2_test = y2[train_index], y2[test_index]

eval_set = [(X2_train, np.ravel(y2_train)), (X2_test, np.ravel(y2_test))]


# Fitting model and validation
reg1 = xgb.XGBRegressor(objective ='reg:squarederror',
                        colsample_bytree = 0.3, 
                        learning_rate = 0.1, 
                        max_depth = 5, 
                        alpha = 10, 
                        n_estimators = 100,
                        random_state = 42)


reg1.fit(X2_train, y2_train, 
         eval_metric = 'rmse', 
         eval_set = eval_set,
         early_stopping_rounds = 10, verbose = 10)


# Regression quality check
print('\033[4mModel performance :: Regression Quality\033[0m')
plt.rcParams['figure.figsize'] = 15, 5 
plt.rcParams['font.size'] = 12

history = reg1.evals_result_
x_axis = range(len(history['validation_0']['rmse']))
plt.plot(x_axis, history['validation_0']['rmse'], label = 'Train')
plt.plot(x_axis, history['validation_1']['rmse'], label = 'Test')
plt.legend(loc = 'best')
plt.ylabel('RMSE')
plt.title('Xgboost model performance')
plt.show()

# imporant features
# Feature importance
plt.rcParams['figure.figsize'] = [15, 10]
plot_importance(reg1)
print('\n')

# confidence check on Close test data
close_confidence = reg1.score(X2_test, y2_test)
print("close_confidence %: ", round(close_confidence*100,2))

# 5 days close price prediction
forecast_pred_1 = reg1.predict(X2_forecast)
#print('\033[4mExpected Open price for next 5 days\033[0m')
#print(round(forecast_pred_1, 2))
#print('\n')

rmse2 = np.sqrt(mean_squared_error(y2_test[:pred_future], forecast_pred_1[:pred_future]))
print('Test RMSE: %.2f' % rmse2)


# assigning date to the predicted values
merge15['close_pred'] = np.nan
last_date = merge15.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day 

for i in forecast_pred_1:
  next_date = datetime.datetime.fromtimestamp(next_unix)
  next_unix += one_day
  merge15.loc[next_date] = [np.nan for _ in range(len(merge15.columns)-1)] + [i]

#create an index of just the date portion of our index (this is the slow step)
ts_days = pd.to_datetime(merge15['close_pred'].index.date)

#create a range of business days over that period
bdays = pd.bdate_range(start=merge15['close_pred'].index[0].date(), end=merge15['close_pred'].index[-1].date())

#Filter the series to just those days contained in the business day range.
merge15['close_pred'] = merge15.close_pred[ts_days.isin(bdays)]
print('\033[4mExpected Close price for next 5 days\033[0m')
round(merge15['close_pred'],2).tail()


# Predicted Open/Close in a dataframe
h_stack = pd.concat([merge16['open_pred'], merge15['close_pred']], axis=1)
h_stack.tail()

# Week, month, year historical analysis

#print('\033[4mNatural Gas continuous contract 1\033[0m')
#print('\033[4mPrint 1st & last 5 rows\033[0m')
NG1 = quandl.get("CHRIS/CME_NG1", authtoken="LSQpgUzwJRoF667ZpzyL") # natural gas continuous contract 1
NG1 = NG1.loc['2000-01-01':,['Open', 'High', 'Low', 'Last', 'Volume']]
NG1.reset_index(level=0, inplace=True)
#print(NG1)
#print('\n')

# feature engineering
#Adding Weeks, Months and Year Columns
#print('\033[4mFeature engineering- Adding Weeks, Months and Year Columns\033[0m')
#print('\033[4mPrint last 5 rows\033[0m')
NG1['week_no'] = NG1['Date'].dt.week
NG1['month_no'] = NG1['Date'].dt.month
NG1['year'] = NG1['Date'].dt.year
NG1['DayofWeek'] = NG1['Date'].dt.dayofweek
#print(NG1.tail())
#print('\n')

#fig = plt.figure(figsize=(15,6))
#plt.plot(NG1[NG1.year >= 2019].groupby('week_no')['Open'].mean().head(52))
#plt.title('Average weekly Henry Hub Futures (contract#1) price in a year')
#plt.show()
#print ('\033[4mThe plot reveals that, the average weekly Henry Hub Futures (Contract#1) price in a year illustrates mainly downward curve.\033[0m')
#print('\n')

# Average Weekly Gas Prices Pivot Table
#print('\033[4mAverage Weekly Gas Prices Pivot Table\033[0m')
#print('\033[4mPrint last 5 rows\033[0m')
pivot_ng = NG1.pivot_table(values = 'Last', columns = ['year'], aggfunc= np.mean,index = ['week_no'])
#print(pivot_ng.tail()) # last 5 rows
#print('\n')

# Average Monthly Gas Prices & Percent Changes
#print('\033[4mAverage Monthly Gas Prices\033[0m')
monthly = NG1.pivot_table(values= 'Last', columns = ['year'],aggfunc = np.mean, index = ['month_no'])
#print(monthly) 
#print('\n')

monthly.loc[0] = monthly.loc[12,:].shift(1)
monthly.fillna(method='ffill', inplace=True) # filling nan values with previous data
monthly = monthly.sort_index()
#print(monthly)
#print('\n')

print('\033[4mAverage monthly gas prices % change\033[0m')
monthly_change = monthly.pct_change()
# dropping the 1st row
monthly_change = monthly_change.drop(monthly_change.index[0])
#print(monthly_change)
#print('\n')

print('\033[4mProbabilities of Monthly Gas Price Decline vs Raise\033[0m')
#print('\033[4mProbability of gas price decline starts to increase by the summer months. It seems that fluctuations in gas price in the initial months of the year is rather high.\033[0m')
monthly_change['raise'] = monthly_change[(monthly_change.iloc
                                                              [:,:]>0)].count(axis=1)/(2020-2000)
monthly_change['decline'] = monthly_change[(monthly_change.iloc
                                                              [:,:]<0)].count(axis=1)/(2020-2000)   
print(monthly_change[['raise', 'decline']])  



#####-------Dashboard--Program---------######

monthly_change = monthly_change.reset_index()
NG = NG.reset_index()
ng =ng.reset_index()
h_stack = h_stack.reset_index()
forecast = df[['Forecast']].copy()
forecast = forecast.reset_index()
print('test')
print(type(perf))

# setting up background and text color
colors = {
    'background': '#111222',
    'text': '#7FDBFF'
}

# setting up font size and alignment
app.layout=html.Div(
    [
     html.Div(
            children='Natural Gas Contract',
            style={
                    'textAlign': 'center',
                    'color': colors['text'],
                    'fontSize':45,
                    'background': '#111222'
                  }
      
            ),
     dcc.Tabs(
         [
          dcc.Tab(
              label='Natural Gas', # tab name
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize': 25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },
              children=[
                html.Div(
                  [
                    dcc.Graph(id='NaturalGas', # plotting natural gas price 
                      figure={'data':[{
                                'x':NG.Date,
                                'open':NG.Open,
                                'high':NG.High,
                                'low':NG.Low,
                                'close':NG.Last,
                                'type':'candlestick'
                                    }],
                              'layout':{
                                      #'title':'Spot Price ->' + str(hh_spot.hh_spot.tail(1)),
                                      'annotations':[{
                                          'text':'Spot Price: ' + ((str(hh_spot.hh_spot.tail(1)).split('Name')[0]).split()[2]) + ' Dollars per MBtu',
                                          'fontSize':20,
                                          'color':colors['text'],
                                          'showarrow':False,
                                          'align':'right',
                                          'x':1,
                                          'y':1,
                                          'xref':'paper',
                                          'yref':'paper'
                                      },
                                      {
                                          'text':'Day: ' + ((str(hh_spot.hh_spot.tail(1)).split('Name')[0]).split()[1]),
                                          'color':colors['text'],
                                          'showarrow':False,
                                          'align':'right',
                                          'x':1,
                                          'y':0.9,
                                          'xref':'paper',
                                          'yref':'paper'
                                      }],
                                      'plot_bgcolor':colors['background'],
                                      'paper_bgcolor':colors['background'],
                                      'font':{
                                          'color':colors['text']
                                              },
                                        
                                        #'marginBottom':'10px'
                                      }})
                
                ],
                style={'width': '100%', 'display': 'inline-block'},
                ),

                html.Div(
                [
                  html.Div(
                    children='  LATEST VALUES',
                    style={
                        'textAlign': 'center',
                        'color': colors['text'],
                        'fontSize':20,
                        'height':35,
                        'background': '#111222',
                        #'marginLeft':'10px'
                        }
            
                        ),
                 
                   html.Div(
                    [
                         dbc.Card(
                          [html.Div("Date",className="card-name",style={'marginTop':'0px','marginLeft':'70px'}),
                          html.P(
                             (str(NG.Date.tail(1)).split('Name')[0]).split()[1],
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 'marginLeft':'70px'
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                                      
                  
                  html.Div(
                    [
                         dbc.Card(
                         # Natural gas current Open price    
                         [html.Div("Open Price",className="card-name",style={'marginTop':'0px'}),
                          html.P(
                             round(NG['Open'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                 
                  html.Div(
                    [
                         dbc.Card(
                          # Natural gas current Close price
                          [html.Div("Close Price",className="card-name",style={'marginTop':'0px'}),

                          html.P(
                             round(NG['Last'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                 
                 html.Div(
                    [
                         dbc.Card(
                          # Natural gas current Adj Close price
                          [html.Div("Adj Close Price",className="card-name",style={'marginTop':'0px'}),

                          html.P(
                            round(NG['Settle'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                                    
                    
                ]
                ),
                html.Div(
                 [
                 
                 html.Div(
                    [
                         dbc.Card(
                          # Natural gas current High price
                         [html.Div("High Price",className="card-name",style={'marginTop':'0px','marginLeft':'335px'}),
                          html.P(
                             round(NG['High'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 'marginLeft':'335px'
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '50%', 'display': 'inline-block','marginBottom':'10px','height':'40%'},
                    ),
                  html.Div(
                    [
                         dbc.Card(
                          # Natural Gas current Low price
                         [html.Div("Low Price",className="card-name",style={'marginTop':'0px'}),
                          html.P(
                             round(NG['Low'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 #'marginLeft':'440px'
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','marginBottom':'10px','height':'40%'},
                    ),
                 html.Div(
                    [
                         dbc.Card(
                          # Natural Gas current Volume
                         [html.Div("Volume",className="card-name",style={'marginTop':'0px'}),
                          html.P(
                             NG['Volume'].tail(1),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 #'marginLeft':'70px'
                             }
                          ),
                           
                           ],

                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','marginBottom':'10px','height':'40%'},
                    ),
                 ]),
                  # Natural gas last 4 weeks Open/Close/High/Low
                  html.Div(
                     [
                      html.Div(
                        children='Last 4 weeks Open/Close price',
                        style={
                          'textAlign': 'center',
                          'color': colors['text'],
                          'fontSize':20,
                          'background': '#111222'
                          }
                        ),
         
                      html.Div(
                          [
                          dash_table.DataTable(
                            id='last_openclose',
                            columns=[{"name": i, "id": i} for i in ng.columns ],
                            data=(ng.tail(20).to_dict('rows')),
                            style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                          'fontWeight':'bold',
                                          'color': colors['text'],
                                          'textAlign':'center'
                                          },
                            style_cell={
                                'backgroundColor': 'rgb(50, 50, 50)',
                                'color': colors['text'],
                                'textAlign':'center'
                                },
                            )
                          ],

                        )
                      ]
                      ),
              ]),
             
          # Buy/Sell plot
          dcc.Tab(
              label='Buy/Sell Direction',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize':25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },   
              children=[
                    dcc.Graph(id='BuysSells',
                        figure={'data':[
                          {'x':df.index,'y':df['Open'],'type':'line','color':'blue','name':'Open price'},
                          {'x':buys.index,'y':df.loc[buys.index]['Open'],'mode':'markers','name':'Buys','marker':{'color':'red'}},
                          {'x':sells.index,'y':df.loc[sells.index]['Open'],'mode':'markers','name':'Sells','marker':{'color':'green'}},
                          
                      ],
                      'layout':{
                        'title':'Buy (Red) and Sell (Green) Signals',
                        'plot_bgcolor':colors['background'],
                        'paper_bgcolor':colors['background'],
                        'font':{
                            'color':colors['text']
                        },
                        'rang':[-0.6,0.6],
                        'marginBottom':'10px',
                        'paddingTop':'10px',
                        'xaxis':{
                            'title':'Date'
                        },
                        'yaxis':{
                            'title':'Price'
                        }
                              }
                        }
                      ),
                      
                      html.Div(
                          [
                           html.Div(
                              [
                                html.Div(
                                  children='Next 5 Days Open/Close price',
                                  style={
                                    'textAlign': 'center',
                                    'color': colors['text'],
                                    'fontSize':20,
                                    'background': '#111222'
                                    }
                                  ),
                                
                                html.Div(
                                    [
                                    html.Div(
                                        [
                                          html.P(["Open Confidence: "+str(round(open_confidence*100,2))+"%"],
                                          style={
                                              'marginTop':'0px',
                                              'marginBottom':'0px',
                                              'marginLeft': '280px',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                          
                                        ],
                                        style={'width': '70%', 'display': 'inline-block','marginTop':'0px','marginBottom':'0px','background': '#111222'}
                                    ),
                                    html.Div(
                                        [
                                          html.P(["Close Confidence: "+str(round(close_confidence*100,2))+"%"],
                                          style={
                                              'marginTop':'0px',
                                              'marginBottom':'0px',
                                              #'marginRight':'40px',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                        ],
                                        style={'width': '30%', 'display': 'inline-block','marginTop':'0px','marginBottom':'0px','background': '#111222'}
                                    )
                                        
                                ]),
                               
                               html.Div(
                                    [
                                    html.Div(
                                        [
                                          html.P(["Open Prediction error %: "+str(round(rmse1,2))],
                                          style={
                                              'marginTop':'0px',
                                              'marginBottom':'0px',
                                              'marginLeft': '280px',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              #'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                          
                                        ],
                                        style={'width': '70%', 'display': 'inline-block','marginTop':'0px','marginBottom':'0px','background': '#111222'}
                                    ),
                                    html.Div(
                                        [
                                          html.P(["Close Prediction error %: "+str(round(rmse2,2))],
                                          style={
                                              'marginTop':'0px',
                                              'marginBottom':'0px',
                                              #'marginRight':'40px',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              #'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                        ],
                                        style={'width': '30%', 'display': 'inline-block','marginTop':'0px','marginBottom':'0px','background': '#111222'}
                                    )
                                        
                                ]),
                              
                                
                  
                                html.Div(
                                    [
                                    dash_table.DataTable(
                                      id='next_openclose',
                                      columns=[{"name": 'Date', "id": 'Date'},
                                              {"name":'Open Prediction','id':'open_pred'},
                                              {"name":'Close Prediction','id':'close_pred'}],
                                      data=(round(h_stack.tail(),2).to_dict('rows')),
                                      style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                                    'fontWeight':'bold',
                                                    'color': colors['text'],
                                                    'textAlign':'center'
                                                    },
                                      style_cell={
                                          'backgroundColor': 'rgb(50, 50, 50)',
                                          'color': colors['text'],
                                          'textAlign':'center'
                                          },
                                      )
                                    ],

                                  )
                                ],
                                style={'width': '50%', 'display': 'inline-block','marginTop':'10px'},
                                ),

                                html.Div(
                                  [
                                    html.Div(
                                        children='Next day Open price direction',
                                        style={
                                            'textAlign': 'center',
                                            'color': colors['text'],
                                            'fontSize':20,
                                            'background': '#111222',
                                            'height':60,
                                            }
                                          ),
                                    html.Div(
                                    [
                                        dash_table.DataTable(
                                            id='open_direction',
                                            columns =(
                                                [
                                                  {'name':'Date', 'id':'Date'},
                                                 {'name':'Open Price Direction','id':'Forecast'}
                                                ]
                                            ),
                                            
                                            data=(forecast.tail().to_dict('records')),
                                            style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                                          'fontWeight':'bold',
                                                          'color': colors['text'],
                                                          'textAlign':'center'},
                                            style_cell={
                                                'backgroundColor': 'rgb(50, 50, 50)',
                                                'color': colors['text'],
                                                'textAlign':'center'
                                            },
                                            
                                          
                                        )
                                    ],
                                    )
                                  ],
                                  style={'width': '48%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'35%'},
                                  ),
                          ]
                      )
                      ]
                  ),
          
           dcc.Tab(
              label='SMA Crossover',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize':25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },   
              children=[
                        html.Div(
                          [
                            dcc.Graph(id='SMACrossover',
                            figure={'data':
                              [
                                {'x':merge14.index,'y':merge14['Last'],'type':'line','name':'Close'},
                                {'x':merge14.index,'y':merge14['sma1'],'type':'line','name':'SMA42'},
                                {'x':merge14.index,'y':merge14['sma2'],'type':'line','name':'SMA252'},
                              ],
                              'layout':{
                                  'title':'Simple Moving Average Crossover',
                                  'annotations':[{
                                      'text':'Sell -> Shorter MA crosses above longer MA (Golden cross)',
                                      'color':colors['text'],
                                      'showarrow':False,
                                      'align':'right',
                                      'x':1,
                                      'y':1,
                                      'xref':'paper',
                                      'yref':'paper'
                                  },
                                  {
                                      'text':'Buy -> Shorter MA crosses below longer MA (Dead Cross)',
                                      'color':colors['text'],
                                      'showarrow':False,
                                      'align':'right',
                                      'x':1,
                                      'y':0.9,
                                      'xref':'paper',
                                      'yref':'paper'
                                  }],
                                  'plot_bgcolor':colors['background'],
                                  'paper_bgcolor':colors['background'],
                                  'font':{
                                      'color':colors['text']
                                  },
                                  'rang':[-0.6,0.6],
                                  'marginBottom':'10px',
                                  'xaxis':{
                                      'title':'Date'
                                  },
                                  'yaxis':{
                                      'title':'Price'
                                  }
                              }
                              })                        
                                              
                                              
                            ]),
                        
                        #cards 0  
                        html.Div(
                        [
                          html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Average Sharpe",className="card-name"),
                                  html.P(
                                    [round(sharpe_ratio,2)*100],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':25}
                              )
                            ],
                            style={'width': '16%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            ),
                                              
                          html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Annualized Sharpe",className="card-name"),
                                  html.P(
                                    [ round(an_sharpe_ratio,2)*100],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':25}
                              )
                            ],
                            style={'width': '17%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            ),
                          html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Avg Daily returns %",className="card-name"),
                                  html.P(
                                    [round(mean_return,2)*100],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':25}
                              )
                            ],
                            style={'width': '16%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            ),
                        html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Avg Volatility %",className="card-name"),
                                  html.P(
                                    [round(return_stdev, 2)*100],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':25}
                              )
                            ],
                            style={'width': '14%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            ),
                        html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Avg Annual return %",className="card-name"),
                                  html.P(
                                    [ round(mean_return * 252,2)*100],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':25}
                              )
                            ],
                            style={'width': '16%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            ),
                                              
                          html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Annual volatility %",className="card-name"),
                                  html.P(
                                    [ round(return_stdev * np.sqrt(252), 2)*100],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':25}
                              )
                            ],
                            style={'width': '15%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            )
                        
                            
                            
                        ]
                        ),
                    #cards 1 
                    html.Div(
                    [
                      html.Div(
                        [
                            dbc.Card(
                              [html.H4("Minimum daily % return",className="card-name"),
                              html.P(
                                [str(round(daily_ret.min(),2)*100)+' ON '+str(daily_ret.idxmin())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':150,
                                    'textAlign': 'center',
                                    'fontSize':25}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                        ),
                                          
                      html.Div(
                        [
                            dbc.Card(
                              [html.H4("Maximum daily % return",className="card-name"),
                              html.P(
                                [str(round(daily_ret.max(),2)*100)+' ON '+str(daily_ret.idxmax())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':150,
                                    'textAlign': 'center',
                                    'fontSize':25}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                        ),
                      html.Div(
                        [
                            dbc.Card(
                              [html.H4("Minimum Close Price",className="card-name"),
                              html.P(
                                [str(round(NG['Last'].min(),2))+' ON '+str(NG['Last'].idxmin())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':150,
                                    'textAlign': 'center',
                                    'fontSize':25}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                        ),
                      html.Div(
                        [
                            dbc.Card(
                              [html.H4("Maximum Close Price",className="card-name"),
                              html.P(
                                [str(round(NG['Last'].max(),2)) +' ON '+ str(NG['Last'].idxmax())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':150,
                                    'textAlign': 'center',
                                    'fontSize':25}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                        ),
                        
                        
                    ]
                    ),
                    #cards 2 
                        html.Div(
                        [
                          html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Probability +/-(1)% change",className="card-name"),
                                  html.P(
                                    [round(100*daily_ret[(daily_ret > -0.01) & (daily_ret < 0.01)].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':20}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            ),
                                              
                          html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Probability +/-(3)% change",className="card-name"),
                                  html.P(
                                    [round(100*daily_ret[(daily_ret > -0.03) & (daily_ret < 0.03)].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':20}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            ),
                          html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Probability +/-(5)% change",className="card-name"),
                                  html.P(
                                    [round(100*daily_ret[(daily_ret > -0.05) & (daily_ret < 0.05)].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':20}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            ),
                          html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Probability > 5% change",className="card-name"),
                                  html.P(
                                    [round(100*daily_ret[daily_ret > 0.05].shape[0] / daily_ret.shape[0], 2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':20}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            ),
                        html.Div(
                            [
                                dbc.Card(
                                  [html.H4("Probability < 5% change",className="card-name"),
                                  html.P(
                                    [round(100*daily_ret[daily_ret < -0.05].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':150,
                                        'textAlign': 'center',
                                        'fontSize':20}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'10px','marginLeft':'10px','height':'40%'},
                            ),
                            
                            
                        ]
                        ),
                        ],
              ),
          
           dcc.Tab(
              label='Drawdown',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize':25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },   
              children=[
                        html.Div(
                          [
                            dcc.Graph(id='DailyDrawdown',
                                      figure={'data':[{
                                            'x':daily_drawdown.index,
                                            'y':daily_drawdown,
                                            'type':'line'
                                        },
                                        
                                        {
                                            'x':min_daily_drawdown.index,
                                            'y':min_daily_drawdown,
                                            'type':'line'
                                        }
                                    ],'layout':{
                                            'title':'Maximum Daily Drawdown',
                                            'plot_bgcolor':colors['background'],
                                            'paper_bgcolor':colors['background'],
                                            'font':{
                                                'color':colors['text']
                                            },
                                            'marginBottom':'10px',
                                            'xaxis':{
                                                'title':'Date'
                                            }            
                                        }}
                                      )                        
                          ],
                          style={'width': '100%', 'display': 'inline-block'},
                        ),

                        html.Div(
                          [
                            html.Div(
                              children='Monthly Price Decline/Raise',
                              style={
                                  'textAlign': 'center',
                                  'color': colors['text'],
                                  'fontSize':20,
                                  'background': '#111222'
                                  }
                              ),
                            html.Div(
                            [
                                dash_table.DataTable(
                                    id='raise_decline',
                                    columns=(
                                    [{"name": 'Month', "id": 'month_no'},
                                    {"name": 'Raise', "id": 'raise'},
                                    {"name": 'Decline', "id": 'decline'},
                                    ]
                                    ),
                                    data=(monthly_change.to_dict('records')),
                                    style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                                  'fontWeight':'bold',
                                                  'color': colors['text'],
                                                  'textAlign':'center'},
                                    style_cell={
                                        'backgroundColor': 'rgb(50, 50, 50)',
                                        'color': colors['text'],
                                        'textAlign':'center'
                                    },
                                    page_action="native",
                                    page_current= 2,
                                    page_size= 4,
                                  
                                )
                            ],
                            )
                    ],
                    style={'width': '100%', 'display': 'inline-block','marginTop':'10px'},
                    ),
                    
                      ]
           )
        
      ])
    ])
         
            

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=True)






Overwriting Sarit_app.py


In [25]:
# Below command generate ngrok link... open link that link after program execution done
get_ipython().system_raw('./ngrok http 8050 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

!python Sarit_app.py





https://786de7dc.ngrok.io
<IPython.core.display.Javascript object>
<IPython.core.display.Javascript object>
<IPython.core.display.Javascript object>
/usr/local/lib/python3.6/dist-packages/ffn/core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
<IPython.core.display.Javascript object>
/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.

Henry Hub Natural Gas Spot Price, Daily (Dollars per Million Btu)
<IPython.core.display.Javascript object>
<class '

KeyboardInterrupt: ignored